In [ ]:
!pip install pm4py

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")

In [ ]:
import os
import pm4py
import graphviz
import pandas as pd
import datetime as dt
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.visualization.dfg import visualizer as dfg_visualization
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.visualization.petri_net import visualizer as pn_visualizer

In [ ]:
sample = pd.read_csv("./sample1.csv")

In [ ]:
sample.head()

In [ ]:
case_list = []
case_number = 1

for ymd in sample["proc_ymd"].unique():
    target = sample[sample["proc_ymd"] == ymd]
    for case_add in range(len(target)):
        case_list.append(case_number)
    case_number += 1

sample["case_id"] = case_list

target = sample[["case_id", "eventtime_kst", "object_description"]].dropna()
eventlog = target.rename(
    columns={
        "case_id": "case:concept:name",
        "eventtime_kst": "time:timestamp",
        "object_description": "concept:name",
        "userid": "org:resource",
    }
)

In [ ]:
target.head()

In [ ]:
# Convert the log to a suitable format for alpha miner
log = log_converter.apply(eventlog)

# Discover the process model using alpha miner
net, initial_marking, final_marking = alpha_miner.apply(log)

# Apply the Petri net visualizer
gviz = pn_visualizer.apply(net, initial_marking, final_marking)

# Convert the DOT source to a string
dot_string = gviz.source

# Render the modified DOT file using Graphviz
graph = graphviz.Source(dot_string)
graph

In [ ]:
eventlog["time:timestamp"] = eventlog["time:timestamp"].apply(
    lambda x: pd.to_datetime(x)
)
log = log_converter.apply(eventlog)

dfg = dfg_discovery.apply(log)
gviz = dfg_visualization.apply(
    dfg, log=log, variant=dfg_visualization.Variants.PERFORMANCE
)

dot_string = gviz.source

# Render the modified Dot file using graphviz
graph = graphviz.Source(dot_string)
graph

In [ ]:
eventlog["time:timestamp"] = eventlog["time:timestamp"].apply(
    lambda x: pd.to_datetime(x)
)
filtered = pm4py.filter_time_range(
    eventlog, dt.datetime(2010, 12, 30), dt.datetime(2010, 12, 31), mode="events"
)


log = log_converter.apply(eventlog)

dfg = dfg_discovery.apply(log)
gviz = dfg_visualization.apply(
    dfg, log=log, variant=dfg_visualization.Variants.FREQUENCY
)

dot_string = gviz.source

# Render the modified DOT file using Graphviz
graph = graphviz.Source(dot_string)
graph

In [ ]:
log_1 = log_converter.apply(log)
filtered = pm4py.get_event_attribute_values(log_1, "concept:name", {"밀크T초등"})

In [ ]:
eventlog["time:timestamp"] = eventlog["time:timestamp"].apply(
    lambda x: pd.to_datetime(x)
)

log = log_converter.apply(eventlog)

dfg = dfg_discovery.apply(log)
gviz = dfg_visualization.apply(
    dfg, log=log, variant=dfg_visualization.Variants.FREQUENCY
)

dot_string = gviz.source

# Render the modified DOT file using Graphviz
graph = graphviz.Source(dot_string)
graph